# Análsis de carpetas de investigación del 2023

Estos datos corresponden al periodo del 01/Enero/2023 al 31/Marzo/2023

In [70]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
from sklearn.impute import KNNImputer
import math
import re

## Entendimiento de los datos

In [50]:
df = pd.read_csv('data/da_2023.csv')

In [51]:
df.columns

Index(['idCarpeta', 'Año_inicio', 'Mes_inicio', 'FechaInicio', 'Delito',
       'Categoria', 'Sexo', 'Edad', 'TipoPersona', 'CalidadJuridica',
       'competencia', 'Año_hecho', 'Mes_hecho', 'FechaHecho', 'HoraHecho',
       'HoraInicio', 'alcaldia_hechos', 'municipio_hechos', 'colonia_datos',
       'fgj_colonia_registro', 'latitud', 'longitud'],
      dtype='object')

In [52]:
df.head(1)

,idCarpeta,Año_inicio,Mes_inicio,FechaInicio,Delito,Categoria,Sexo,Edad,TipoPersona,CalidadJuridica,...,Mes_hecho,FechaHecho,HoraHecho,HoraInicio,alcaldia_hechos,municipio_hechos,colonia_datos,fgj_colonia_registro,latitud,longitud
0,9361878,2023,Enero,2023-01-01,LESIONES CULPOSAS POR TRANSITO VEHICULAR EN CO...,DELITO DE BAJO IMPACTO,NaN,NaN,MORAL,OFENDIDO,...,Diciembre,2022-12-31,22:08:00,00:09:00,COYOACAN,NaN,INSURGENTES CUICUILCO,INSURGENTES CUICUILCO,19.305532,-99.186308


In [53]:
df.dtypes

idCarpeta                 int64
Año_inicio                int64
Mes_inicio               object
FechaInicio              object
Delito                   object
Categoria                object
Sexo                     object
Edad                    float64
TipoPersona              object
CalidadJuridica          object
competencia              object
Año_hecho               float64
Mes_hecho                object
FechaHecho               object
HoraHecho                object
HoraInicio               object
alcaldia_hechos          object
municipio_hechos         object
colonia_datos            object
fgj_colonia_registro     object
latitud                 float64
longitud                float64
dtype: object

In [54]:
df.nunique()

idCarpeta               61333
Año_inicio                  1
Mes_inicio                  3
FechaInicio                90
Delito                    255
Categoria                  16
Sexo                        2
Edad                      100
TipoPersona                 2
CalidadJuridica             5
competencia                 3
Año_hecho                  39
Mes_hecho                  12
FechaHecho               1434
HoraHecho                1441
HoraInicio               1455
alcaldia_hechos            18
municipio_hechos           91
colonia_datos            1655
fgj_colonia_registro     1538
latitud                 57654
longitud                57653
dtype: int64

In [55]:
df["Año_hecho"].describe()

count    64337.000000
mean      2022.731616
std          1.271916
min       1962.000000
25%       2023.000000
50%       2023.000000
75%       2023.000000
max       2023.000000
Name: Año_hecho, dtype: float64

### Tenemos delitos cometidos desde el año **1962** a los cuáles se les abrió carpetas de investigación hasta el año **2023**

In [56]:
df["Año_hecho"].unique()

array([2022., 2023., 2021., 2018., 2019., 2020., 2013., 1998., 2007.,
       2017., 2014., 2015., 2010., 2011.,   nan, 2003., 2016., 2005.,
       2000., 2006., 1993., 1995., 2012., 2002., 2008., 1969., 1987.,
       2009., 1967., 1990., 1999., 2004., 1994., 1974., 1976., 1985.,
       1962., 1991., 1972., 2001.])

In [57]:
df["Categoria"].unique()

array(['DELITO DE BAJO IMPACTO',
       'ROBO A TRANSEUNTE EN VÍA PÚBLICA CON Y SIN VIOLENCIA',
       'ROBO DE VEHÍCULO CON Y SIN VIOLENCIA', 'HECHO NO DELICTIVO',
       'LESIONES DOLOSAS POR DISPARO DE ARMA DE FUEGO', 'VIOLACIÓN',
       'ROBO A NEGOCIO CON VIOLENCIA',
       'ROBO A REPARTIDOR CON Y SIN VIOLENCIA', 'HOMICIDIO DOLOSO',
       'ROBO A PASAJERO A BORDO DE MICROBUS CON Y SIN VIOLENCIA',
       'ROBO A PASAJERO A BORDO DEL METRO CON Y SIN VIOLENCIA',
       'ROBO A PASAJERO A BORDO DE TAXI CON VIOLENCIA',
       'ROBO A CUENTAHABIENTE SALIENDO DEL CAJERO CON VIOLENCIA',
       'ROBO A CASA HABITACIÓN CON VIOLENCIA',
       'ROBO A TRANSPORTISTA CON Y SIN VIOLENCIA', 'SECUESTRO'],
      dtype=object)

In [58]:
df.loc[df["Año_hecho"]==1962]

,idCarpeta,Año_inicio,Mes_inicio,FechaInicio,Delito,Categoria,Sexo,Edad,TipoPersona,CalidadJuridica,...,Mes_hecho,FechaHecho,HoraHecho,HoraInicio,alcaldia_hechos,municipio_hechos,colonia_datos,fgj_colonia_registro,latitud,longitud
44757,9408996,2023,Marzo,2023-03-06,AMENAZAS,DELITO DE BAJO IMPACTO,Masculino,60.0,FISICA,VICTIMA Y DENUNCIANTE,...,Septiembre,1962-09-11,13:45:00,18:39:00,COYOACAN,NaN,DEL CARMEN,DEL CARMEN,19.351477,-99.158951


In [59]:
df["competencia"].unique()

array(['FUERO COMUN', 'HECHOS NO DELICTIVOS', 'INCOMPETENCIAS'],
      dtype=object)

### Resumen del entendimiento de los datos

1. Existen carpetas de investigación sobre delitos cometidos desde 1962, abiertas hasta el 2023.

2. Tenemos **16 categorías de delitos**.

3. Hay **5 tipos de calidades jurídicas**.

4. Describiendo los **3 competencias de delitos**:
    
    * Fuero común: Aquellos delitos que afectas directamente a la persona.
    * Hechos no delictivos: No son delitos.
    * Incompetencias: La persona es incapaz de ser juzgada o de ser responsable de sus acciones debido a condiciones físicas o mentales.
    
    
    
5. En la columna de alcaldías estamos considerando un **extra llamado 'fuera de la ciudad de méxico'**

6. Tenemos **255 tipos de delitos**.

---

#### Recomendaciones en el proceso de limpieza

- Eliminar las siguientes columnas:
    
        1. idCarpeta
        
        
- Renombrar las siguientes columnas:

        1. Año_hecho
        2. Año_inicio
        3.fgj_colonia_registro
        

- Cambiar el tipo de dato a la columna edades y al año_hecho

- Estandarizar los nombres de las columnas a minusculas

## Limpieza

1. [X] Eliminar columnas que se relacionen con el tiempo en que se hizo la denuncia
2. [X] Estandarizar a minusculas las columnas restantes
3. [X] Renombrar columnas a 1 sola palabra
4. [X] Eliminar la palabra hecho de todas las columnas que la contengan
5. [X] Convertir a formato de fecha si corresponde
6. [ ] Rellenar los años NAN con **imputación**
7. [ ] Cambiar a enteros los años
8. [ ] Si existe un tipo de dato de "hora", cambiarlo en la que corresponda
9. [ ] Separar en 2 DataFrames teniendo en cuenta el tipo de persona
10. [ ] Del dataframe de personas morales, eliminar las columnas de sexo, edad, tipopersona
11. [ ] En la columna de edades de personas físicas, llenar los NA con algún **algoritmo de imputación**


... Si hay más, apúntelos aquí

### Paso 1

In [ ]:
df.drop(
        columns=["fgj_colonia_registro", "idCarpeta", "Año_inicio", "Mes_inicio", "FechaInicio", "HoraInicio"], 
        inplace=True
        )

### Paso 2

In [66]:
df.rename(columns= lambda x: x.lower(), inplace=True)

### Paso 3

In [106]:
df.rename(columns = 
         {
             'tipopersona': 'persona',
             'calidadjuridica': 'calidad',
             'colonia_datos': 'colonia'
         }, 
        inplace=True)

### Paso 4

In [93]:
def find(name):
    return re.sub(r'_?hechos?$', '', name)

df.columns = df.columns.to_frame()[0].agg(find)

### Paso 5

In [97]:
pd.to_datetime(df.fecha)

0       2022-12-31
1       2022-12-31
2       2022-12-31
3       2022-12-31
4       2022-12-31
           ...    
64339   2023-03-31
64340   2023-03-30
64341   2023-03-31
64342   2023-03-31
64343   2023-03-31
Name: fecha, Length: 64344, dtype: datetime64[ns]

### Paso 6: Rellenar los años NAN con **imputación**

Tenemos varios métodos de imputación, por ejemplo obtener medias, medianas, modas, etc.

También unos más complejos como el KNN imputation, extrapolación

### Paso 7

### Paso 8

In [98]:
#df_fisica = df.query('tipopersona == "FISICA"')
#df_moral = df.query('tipopersona == "MORAL"')

In [99]:
# df_moral.drop(labels=["sexo", "edad"], axis=1)

In [100]:
# df_fisica.mes_hecho.unique()

In [101]:
#df_fisica.loc[:, "mes_hecho"].fillna(value="Sin informacion", inplace=True)

In [102]:
#df_fisica.mes_hecho.unique()

In [103]:
# df_fisica.año_inicio.unique()

In [104]:
# df_fisica.año_inicio.fillna(value=2023, inplace=True)